In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

## Reading datas

anime csv

In [2]:
anime = pd.read_csv("anime.csv")
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


deleting ova, extra, etc, Only selecting TV Shows and Movie  

In [3]:
print("========= Before =========")
print(anime.shape)
anime = anime[(anime['type'] == 'TV') | (anime['type'] == 'Movie')]
print("========= After =========")
print(anime.shape)

========= Before =========
(12294, 7)
========= After =========
(6135, 7)


Because my pc gonna blow up and we only need the relevant animes so we are gonna limit the data to only 75% (picked by the most famous)

In [4]:
limit = anime['members'].quantile(0.75)
anime = anime[(anime['members'] >= limit)]
print("========= Anime scale =========")
print(anime.shape)

========= Anime scale =========
(1534, 7)


Read the rating.csv

In [5]:
rating = pd.read_csv("rating.csv")
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [6]:
print("========= Rating scale =========")
rating.shape

========= Rating scale =========


(7813742, 3)

## Preprocessing data

Changing data rating that has no values (-1) to be NaN

In [7]:
rating.loc[rating.rating == -1, 'rating'] = np.NaN
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


index for anime name

In [8]:
index = pd.Series(anime.index, index=anime.name)
index.head()

name
Kimi no Na wa.                      0
Fullmetal Alchemist: Brotherhood    1
Gintama°                            2
Steins;Gate                         3
Gintama&#039;                       4
dtype: int64

merge

In [9]:
joined = anime.merge(rating, how='inner', on='anime_id')
joined.head()

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5.0
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10.0
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10.0
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10.0
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,NaN


^ 4 will be error no matter what cause they didn't give any ratings

In [10]:
joined = joined[['user_id', 'name', 'rating_y']]
pivot = pd.pivot_table(joined, index='name', columns='user_id', values='rating_y')
pivot.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73508,73509,73510,73511,73512,73513,73514,73515,73516,73517
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
pivot.shape

(1490, 69225)

In [12]:
pivot.dropna(axis=1, how='all', inplace=True)
pivot.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73508,73509,73510,73511,73512,73513,73514,73515,73516,73517
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pivot.shape

(1490, 69225)

Pivot subtracts the mean of the row from each element of the row. (Nomalization)

In [14]:
norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
norm.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73508,73509,73510,73511,73512,73513,73514,73515,73516,73517
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert NaN to zeros

In [15]:
norm.fillna(0, inplace=True)
norm.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73508,73509,73510,73511,73512,73513,73514,73515,73516,73517
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


convert into dataframe to make it easier, Calculate the similarity with cosine similarity

In [16]:
item_sim_df = pd.DataFrame(cosine_similarity(norm, norm), index=norm.index, columns=norm.index)
item_sim_df.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,1.000000,0.032753,0.026011,0.036676,0.038012,0.046374,0.047874,0.009425,0.056224,0.040200,...,0.048061,0.036628,0.045698,0.031965,0.073987,0.079183,0.008384,0.035791,0.049389,0.047023
.hack//Roots,0.032753,1.000000,0.289985,0.315053,0.074391,0.061119,0.027616,0.006698,0.027652,0.031912,...,0.043802,0.034236,0.066003,0.004396,0.038025,0.030098,0.049731,0.038284,0.040426,0.031665
.hack//Sign,0.026011,0.289985,1.000000,0.269825,0.058850,0.050163,0.033374,0.014148,0.035317,0.031722,...,0.038653,0.033260,0.056693,0.005344,0.049085,0.050667,0.067435,0.055709,0.031301,0.033710
.hack//Tasogare no Udewa Densetsu,0.036676,0.315053,0.269825,1.000000,0.047875,0.053741,0.027466,0.009595,0.040570,0.023247,...,0.038657,0.034183,0.063091,0.000434,0.044590,0.036658,0.072843,0.048902,0.041610,0.038522
07-Ghost,0.038012,0.074391,0.058850,0.047875,1.000000,0.122489,0.040180,0.021060,0.051344,0.035019,...,0.094213,0.017888,0.129566,0.007980,0.042737,0.036837,0.041351,0.054222,0.066368,0.042567


In [17]:
def get_similar_anime(anime_name):
    if anime_name not in norm.index:
        return None, None
    else:
        sim_animes = item_sim_df.sort_values(by=anime_name, ascending=False).index[1:]
        sim_score = item_sim_df.sort_values(by=anime_name, ascending=False).loc[:, anime_name].tolist()[1:]
        return sim_animes, sim_score

test

In [18]:
animes, score = get_similar_anime("Steins;Gate")
for x,y in zip(animes[:10], score[:10]):
    print("{} with similarity of {}".format(x, y))

Steins;Gate Movie: Fuka Ryouiki no Déjà vu with similarity of 0.24336781483924164
Psycho-Pass with similarity of 0.1841425084628351
Code Geass: Hangyaku no Lelouch R2 with similarity of 0.18246014048788292
Fate/Zero 2nd Season with similarity of 0.18113560310020757
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. with similarity of 0.17588179421245337
Fate/Zero with similarity of 0.17547220214977552
Fullmetal Alchemist: Brotherhood with similarity of 0.168233519502045
Code Geass: Hangyaku no Lelouch with similarity of 0.16785600931461606
Angel Beats! with similarity of 0.1634834913449394
Nisemonogatari with similarity of 0.15594071809613452


predict the rating of anime x by user y

In [19]:
def predict_rating(user_id, anime_name, max_neighbor=5):
    animes, scores = get_similar_anime(anime_name)
    anime_arr = np.array([x for x in animes])
    
    sim_arr = np.array([x for x in scores])

    # select only the anime that has already rated by user x
    filtering = norm[user_id].loc[anime_arr] != 0
    
    #calculate the score
    score = np.dot(sim_arr[filtering][:max_neighbor], pivot[user_id].loc[anime_arr[filtering][:max_neighbor]]) / np.sum(sim_arr[filtering][:max_neighbor])
    
    return score

test

In [20]:
predict_rating(3, "Steins;Gate")

9.445606747333231

In [21]:
predict_rating(3, "Cowboy Bebop")

8.973493240584913

Recommend the top 5 animes

In [22]:
def get_recommendation(user_id, n_anime=5):
    predicted_rating = np.array([])
    for _anime in norm.index:
        predicted_rating = np.append(predicted_rating, predict_rating(user_id, _anime))
    
    # if rated then no recommend 🗿
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':norm.index})
    filtering = (norm[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)

    # recommend n_anime
    return anime.loc[index.loc[temp.name[:n_anime]]]

In [23]:
get_recommendation(1)

,anime_id,name,genre,type,episodes,rating,members
1356,6895,Hakuouki,"Action, Drama, Historical, Josei, Samurai, Sup...",TV,12,7.57,90612
50,7785,Yojouhan Shinwa Taikei,"Mystery, Psychological, Romance",TV,11,8.65,122531
833,3713,Jigoku Shoujo Mitsuganae,"Mystery, Psychological, Supernatural",TV,26,7.81,58993
4024,18247,IS: Infinite Stratos 2,"Action, Comedy, Harem, Mecha, Sci-Fi",TV,12,6.82,146979
942,5507,Senjou no Valkyria,"Action, Military, Romance",TV,26,7.76,77591


In [24]:
get_recommendation(3)

,anime_id,name,genre,type,episodes,rating,members
78,17389,Kingdom 2nd Season,"Action, Historical, Military, Seinen",TV,39,8.57,31234
44,19647,Hajime no Ippo: Rising,"Comedy, Drama, Shounen, Sports",TV,25,8.68,66756
20,263,Hajime no Ippo,"Comedy, Drama, Shounen, Sports",TV,75,8.83,157670
32,5258,Hajime no Ippo: New Challenger,"Comedy, Drama, Shounen, Sports",TV,26,8.75,88995
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855


this code block is error cause user number 4 doesn't provide any rating

In [25]:
get_recommendation(4)

KeyError: 4

# Note:  
- Coldstart
- Big Dataset

# Case  
### User
Nama: Jamal  
Hobi: Nonton Anime  
UserID: 73517

Jamal sangat menyukai anime, tiap hari dia nonton anime. Beberapa anime kesukaan jamal adalah:
|Anime_id|Anime|Rating|
|--|--|--|
|26055|JoJo season 2|10|
|20899|JoJo no Kimyou na Bouken: Stardust Crusaders|10|
|28977|Gintama|9|
|2759|Evangelion: 1.0 You Are (Not) Alone|9|
|30279|Yuru Yuri San☆Hai!|10|

In [26]:
get_recommendation(73517)

,anime_id,name,genre,type,episodes,rating,members
11,28851,Koe no Katachi,"Drama, School, Shounen",Movie,1,9.05,102733
2457,12979,Naruto SD: Rock Lee no Seishun Full-Power Ninden,"Action, Comedy, Parody",TV,51,7.26,29989
2115,32245,Kuromukuro,"Action, Mecha, Sci-Fi",TV,26,7.35,46323
37,31757,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",Movie,1,8.73,34347
520,12815,Shirokuma Cafe,"Comedy, Josei, Slice of Life",TV,50,8.01,44668
